<a href="https://colab.research.google.com/github/MuindeEsther/BootcampProjects/blob/main/RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **WEEK 2**

##**PROJECT 2**

### **RFM ANALYSIS**

Recency, Frequency, and Monetary is a marketing technique used to quantitavely rank and group customers based on the recency, frequency and monetary total of their recent transactions to identify the best customers & perform targeted marketed campaigns.

Task
Perform RFM analysis and develop customer segments based on their RFM Scores
The analysis should provide insights into customer behaviour & identification of high-value customers, at-risk customers, & potential opportunities for personalized marketing campaigns



In [3]:
# Load important libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image


In [4]:
# Read data
df = pd.read_csv('/content/drive/MyDrive/Bootcamp data files/rfm_data.csv')
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [5]:
df.info


<bound method DataFrame.info of      CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0          8814   2023-04-11             943.31          Product C   890075   
1          2188   2023-04-11             463.70          Product A   176819   
2          4608   2023-04-11              80.28          Product A   340062   
3          2559   2023-04-11             221.29          Product A   239145   
4          9482   2023-04-11             739.56          Product A   194545   
..          ...          ...                ...                ...      ...   
995        2970   2023-06-10             759.62          Product B   275284   
996        6669   2023-06-10             941.50          Product C   987025   
997        8836   2023-06-10             545.36          Product C   512842   
998        1440   2023-06-10             729.94          Product B   559753   
999        4759   2023-06-10             804.28          Product D   467544   

     Location  
0  

In [6]:
df.info(['ProductInformation'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [7]:
df.describe()

,CustomerID,TransactionAmount,OrderID
count,1000.000000,1000.00000,1000.000000
mean,5554.789000,513.67781,554071.398000
std,2605.014863,286.09870,264695.448814
min,1011.000000,12.13000,100096.000000
25%,3273.000000,257.12000,313152.000000
50%,5538.000000,523.56500,564671.500000
75%,7821.750000,759.86000,783052.250000
max,9991.000000,999.44000,999695.000000


In [8]:
#Lets tansform customerid to object same case to orderid
df['CustomerID'] = df['CustomerID'].astype(str)
df['OrderID'] = df['OrderID'].astype(str)

In [9]:
print(df.dtypes)

CustomerID             object
PurchaseDate           object
TransactionAmount     float64
ProductInformation     object
OrderID                object
Location               object
dtype: object


In [10]:
product_types = df['ProductInformation'].unique()
print(product_types)

['Product C' 'Product A' 'Product B' 'Product D']


## Data Preprocessing


* Hndle missing values, outliers and any inconsistencies in the data
* Calculate the recency, frequency, and monetary value for each customer based on their transaction history.



In [11]:
# Check for missing values, outliers and any inconsistencies
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

In [12]:
df.describe()

,TransactionAmount
count,1000.00000
mean,513.67781
std,286.09870
min,12.13000
25%,257.12000
50%,523.56500
75%,759.86000
max,999.44000


In [13]:
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


In [14]:
df['PurchaseDate'].dtype

dtype('O')

**Calculating RFM Values**

In [15]:
from datetime import datetime

# Convert PurchaseDate to datetime
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

# Calculate Recency
current_date = datetime.now().date()
df['Recency'] = (current_date - df['PurchaseDate'].dt.date).dt.days

# Calculate Frequency
frequency_df = df.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_df.rename(columns={'OrderID': 'Frequency'}, inplace=True)
df = df.merge(frequency_df, on='CustomerID', how='left')

# Calculate Monetary value
monetary_df = df.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_df.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
df = df.merge(monetary_df, on='CustomerID', how='left')


In [16]:
print(df.head())

  CustomerID PurchaseDate  TransactionAmount ProductInformation OrderID  \
0       8814   2023-04-11             943.31          Product C  890075   
1       2188   2023-04-11             463.70          Product A  176819   
2       4608   2023-04-11              80.28          Product A  340062   
3       2559   2023-04-11             221.29          Product A  239145   
4       9482   2023-04-11             739.56          Product A  194545   

   Location  Recency  Frequency  MonetaryValue  
0     Tokyo      183          1         943.31  
1    London      183          1         463.70  
2  New York      183          1          80.28  
3    London      183          1         221.29  
4     Paris      183          1         739.56  


## RFM Score Calculation


We will calculate RFM scores for each customer

In [17]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1] # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5] # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for high monetary value

# Calculate RFM scores
df['RecencyScore'] = pd.cut(df['Recency'], bins=5, labels=recency_scores)
df['FrequencyScore'] = pd.cut(df['Frequency'], bins=5, labels=frequency_scores)
df['MonetaryScore'] = pd.cut(df['MonetaryValue'], bins=5, labels=monetary_scores)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   object        
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   object        
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   MonetaryValue       1000 non-null   float64       
 9   RecencyScore        1000 non-null   category      
 10  FrequencyScore      1000 non-null   category      
 11  MonetaryScore       1000 non-null   category      
dtypes: category(3), datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 81.7+ KB


In [19]:
# Convert RFM scores to numeric type
df['RecencyScore'] = df['RecencyScore'].astype(int)
df['FrequencyScore'] = df['FrequencyScore'].astype(int)
df['MonetaryScore'] = df['MonetaryScore'].astype(int)

## RFM Segmentation
### Quantile-Based Segmentation
Divide customers into quartiles or quintiles based on their RFM scores. This creates predefine segments like"High-Value", "Mid-Value", and "Low-Value" customers

In [20]:
# Combine individual score
df['RFM_Score'] = df['RecencyScore'] + df['FrequencyScore'] + df['MonetaryScore']

segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
df['Segment Value'] = pd.qcut(df['RFM_Score'], q=3, labels=segment_labels)


In [21]:
print(df.head())

  CustomerID PurchaseDate  TransactionAmount ProductInformation OrderID  \
0       8814   2023-04-11             943.31          Product C  890075   
1       2188   2023-04-11             463.70          Product A  176819   
2       4608   2023-04-11              80.28          Product A  340062   
3       2559   2023-04-11             221.29          Product A  239145   
4       9482   2023-04-11             739.56          Product A  194545   

   Location  Recency  Frequency  MonetaryValue  RecencyScore  FrequencyScore  \
0     Tokyo      183          1         943.31             1               1   
1    London      183          1         463.70             1               1   
2  New York      183          1          80.28             1               1   
3    London      183          1         221.29             1               1   
4     Paris      183          1         739.56             1               1   

   MonetaryScore  RFM_Score Segment Value  
0              2        

In [22]:
# RFM Segmentation Distribution
segments_counts = df['Segment Value'].value_counts().reset_index()
segments_counts.columns = ['Segment Value', 'Count']

pastel_colors = px.colors.qualitative.Pastel

# Bar chart
fig_segment_dist = px.bar(segments_counts, x='Segment Value', y='Count',
                          color='Segment Value', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')
fig_segment_dist.update_layout(xaxis_title = 'RFM Value Segment',
                               yaxis_title='Count',
                               showlegend=False)
fig_segment_dist.show()

RFM Customer segments

Let's create and analyze RFM Customer Segments that are broader classifications based on the RFM scores.i.e champions, Potential Loyalists and Can't Lose


In [23]:
# Create a new column for RFM Customer Segments
df['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
df.loc[df['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
df.loc[(df['RFM_Score'] >= 6) & (df['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
df.loc[(df['RFM_Score'] >= 5) & (df['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
df.loc[(df['RFM_Score'] >= 4) & (df['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
df.loc[(df['RFM_Score'] >= 3) & (df['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Print the updated data with RFM segments
print(df[['CustomerID', 'RFM Customer Segments']])

    CustomerID RFM Customer Segments
0         8814            Can't Lose
1         2188                  Lost
2         4608                  Lost
3         2559                  Lost
4         9482            Can't Lose
..         ...                   ...
995       2970   Potential Loyalists
996       6669   Potential Loyalists
997       8836   Potential Loyalists
998       1440   Potential Loyalists
999       4759   Potential Loyalists

[1000 rows x 2 columns]


### RFM Analysis
Let's analyze the distribution of customers across different RFM customer segments within each value

In [24]:
df.head(10)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Segment Value,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,183,1,943.31,1,1,2,4,Low-Value,Can't Lose
1,2188,2023-04-11,463.70,Product A,176819,London,183,1,463.70,1,1,1,3,Low-Value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,183,1,80.28,1,1,1,3,Low-Value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,183,1,221.29,1,1,1,3,Low-Value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,183,1,739.56,1,1,2,4,Low-Value,Can't Lose
5,8483,2023-04-11,375.23,Product C,691194,Paris,183,1,375.23,1,1,1,3,Low-Value,Lost
6,8317,2023-04-11,272.56,Product B,826847,New York,183,2,974.88,1,3,3,7,Mid-Value,Potential Loyalists
7,6911,2023-04-11,433.33,Product C,963918,Tokyo,183,1,433.33,1,1,1,3,Low-Value,Lost
8,8993,2023-04-12,16.55,Product D,112426,New York,182,1,16.55,1,1,1,3,Low-Value,Lost
9,3519,2023-04-12,464.63,Product C,139726,New York,182,1,464.63,1,1,1,3,Low-Value,Lost


In [25]:
df.info

<bound method DataFrame.info of     CustomerID PurchaseDate  TransactionAmount ProductInformation OrderID  \
0         8814   2023-04-11             943.31          Product C  890075   
1         2188   2023-04-11             463.70          Product A  176819   
2         4608   2023-04-11              80.28          Product A  340062   
3         2559   2023-04-11             221.29          Product A  239145   
4         9482   2023-04-11             739.56          Product A  194545   
..         ...          ...                ...                ...     ...   
995       2970   2023-06-10             759.62          Product B  275284   
996       6669   2023-06-10             941.50          Product C  987025   
997       8836   2023-06-10             545.36          Product C  512842   
998       1440   2023-06-10             729.94          Product B  559753   
999       4759   2023-06-10             804.28          Product D  467544   

     Location  Recency  Frequency  Monetary

Distibution of customer segments interms of different locations

In [26]:
df.groupby(['Location',])['RFM Customer Segments'].count()

Location
London      246
New York    247
Paris       229
Tokyo       278
Name: RFM Customer Segments, dtype: int64

In [27]:
# Stacked bar chart
segment_dist = df.groupby(['Segment Value', 'RFM Customer Segments'])['CustomerID'].count().reset_index()
fig = px.bar(segment_dist, x='Segment Value', y='CustomerID', color='RFM Customer Segments',
             title='Customer Distribution by RFM Segments within Value Segments',
             labels={'Segment Value': 'Segment Value', 'CustomerID': 'Customer Count', 'RFM Customer Segments': 'RFM Segments'})
fig.show()

In [28]:
segment_product_counts = df.groupby(['Segment Value', 'RFM Customer Segments']).size().reset_index(name='Count')
segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

# Define custom colors
custom_colors = {
    'High-Value':'#ADD8E6',
    'Mid-Value':'#800080',
    'Low-Value':'#FFC0CB',
}

fig_treemap = px.treemap(segment_product_counts,
                         path=['Segment Value', 'RFM Customer Segments'],
                         values='Count',
                         color='Segment Value',
                         color_discrete_map=custom_colors,
                         title='RFM Customer Segments by Value')
fig_treemap.show()

Let's analyze the distribution of RFM Values within the Champions segment


In [29]:
#Filter the dat to include only the customers in the Champions segment
champions_segment = df[df['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)
fig.show()

Let's analyze the correlation of the recency, frequency, and monetary scoes within the champions segment.

In [30]:
correlation_matrix = champions_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

print(correlation_matrix)

# Heatmap
fig_heatmap = px.imshow(correlation_matrix, x=['RecencyScore', 'FrequencyScore', 'MonetaryScore'],
                        y=['RecencyScore', 'Frequency', 'MonetaryScore'],
                        color_continuous_scale='Viridis', title='Correlation Heatmap (Champions)')
# Show Heatmap
fig_heatmap.show()

                RecencyScore  FrequencyScore  MonetaryScore
RecencyScore        1.000000       -0.571727      -0.474715
FrequencyScore     -0.571727        1.000000       0.390657
MonetaryScore      -0.474715        0.390657       1.000000


Let's look at the number of customers in all the segements

In [31]:
# Group by RFM Segments and count the number of customers in each segment
customer_seg_counts = df['RFM Customer Segments'].value_counts().reset_index()
customer_seg_counts.columns = ['RFM Customer Segments', 'Count']

# Define custom colors for each segment
custom_colors = {
    'Champions': 'blue',
    'Potential Loyalists':'green',
    'At Risk Customers':'yellow',
    'Can\'t Lose':'orange',
    'Lost':'red'
}

# Create a bar chart
fig_bar = px.bar(customer_seg_counts, x='RFM Customer Segments', y='Count',
                 color='RFM Customer Segments', color_discrete_map=custom_colors,
                 title='Number of Customers in Each RFM Segment',
                 labels={'RFM Customer Segments':'RFM Segments', 'Count':'Customer Count'})
fig_bar.show()

Let's have a look at the recency, frequency, and monetary scores of all segments

In [32]:
# Lets create the bar chart first without averaging the scores
# Melt the DataFrame to have the score in a single column for grouping
df_melted = pd.melt(df, id_vars=['RFM Customer Segments'], var_name='RFM Category', value_name='Score')

# Create a grouped bar chart
fig_grouped = px.bar(df_melted, x='RFM Customer Segments', y='Score', color='RFM Category',
                     title='RFM Scores by Segment',
                     labels={'RFM Customer Segemnts':'RFM Segments', 'Score':'RFM Score'},
                     category_orders={'RFM Category':['RecencyScore', 'FrequencyScore', 'MonetaryScore']})
fig_grouped.show()

Our bar graph is redudant , lets average the scores and create

In [36]:
# Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = df.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore', 'MonetaryScore'].mean().reset_index()

# Create an interactive grouped bar chart
fig = px.bar(segment_scores, x='RFM Customer Segments',
             y=['RecencyScore', 'FrequencyScore', 'MonetaryScore'],
             title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
             labels={'variable': 'RFM Score'},
             color_discrete_sequence=px.colors.qualitative.Set1)

# Customize the layout
fig.update_layout(
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
)


# Show the interactive chart
fig.show()

<ipython-input-36-8f8cc3b4f10a>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

